### 3-3．모델작성
#### 데이터의 리드와 확인

In [10]:
import pandas as pd
d_df_new = pd.read_csv('b-prep.csv', sep=',')

d_df_new.head()

,age,job2,number_yoy2,member2,period2,budget2,pre_d,Hawaii,Ho Chi Minh,Hong Kong,...,Oceania,South America,fall,spring,summer,winter,bicycle,plane,ship,walking
0,20,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,20,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,20,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,20,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,1,0,0,0
4,20,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0


#### 불균형 데이터의 균형화

In [2]:
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.8.0
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: None
Author-email: None
License: MIT
Location: /opt/conda/lib/python3.8/site-packages
Requires: numpy, joblib, scikit-learn, scipy
Required-by: 


In [3]:
!pip list

Package                       Version
----------------------------- -------------------
alembic                       1.5.2
anyio                         2.0.2
appdirs                       1.4.4
argon2-cffi                   20.1.0
async-generator               1.10
attrs                         20.3.0
Babel                         2.9.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.1
beautifulsoup4                4.9.3
bleach                        3.2.3
blinker                       1.4
bokeh                         2.2.3
Bottleneck                    1.3.2
brotlipy                      0.7.0
cached-property               1.5.1
certifi                       2020.12.5
certipy                       0.1.3
cffi                          1.14.4
chardet                       4.0.0
click                         7.1.2
cloudpickle                   1.6.0
cmake                         3.18.4.post1
conda                         4.9.2
conda-package-handling        1.7.2
c

In [4]:
!pip install imbalanced-learn

In [11]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

X = np.array(d_df_new.drop('pre_d', axis=1))
Y = np.array(d_df_new['pre_d'])

print(np.sum(Y == 1), np.sum(Y == 0))

sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

print(np.sum(Y == 1), np.sum(Y == 0))

1769 967
967 967


**----여기서부터 추가----**

In [14]:
from sklearn.feature_selection import SelectKBest

# 특징량을 5개 선택
selector = SelectKBest(k=5) 
selector.fit(X, Y)
mask = selector.get_support()

# 어느 변수를 선택하였는지 확인
print(d_df_new.drop('pre_d', axis=1).columns)
print(mask)

Index(['age', 'job2', 'number_yoy2', 'member2', 'period2', 'budget2', 'Hawaii',
       'Ho Chi Minh', 'Hong Kong', 'Jeju', 'LA', 'London', 'New York', 'Paris',
       'Prague', 'Singapore', 'Sydney', 'Tokyo', 'Vancouver', 'unknown',
       'email', 'letter', 'phone', 'Beijing duck', 'Chicken Rice',
       'Croissant Sandwich', 'Fish and Chips', 'Gelato', 'Kebob',
       'Masaman Curry', 'Pad Thai', 'Rice Noodles', 'Som Tom', 'bulgogi',
       'green curry', 'ice cream', 'kimchi', 'lasagne', 'nasi goreng', 'ramen',
       'rendang', 'satay', 'sushi', 'tomyumgoong', 'Apple', 'Banana', 'Coffee',
       'Grapefruit', 'Kiwi', 'Mango', 'Orange', 'Strawberry', 'Tomato',
       'Watermelon', 'Africa', 'Antarctica', 'Asia', 'Europe', 'North America',
       'Oceania', 'South America', 'fall', 'spring', 'summer', 'winter',
       'bicycle', 'plane', 'ship', 'walking'],
      dtype='object')
[ True False False  True False False False False False False False False
 False False False False False Fa

In [16]:
# 선택한 특징량의 열만 취득
X_sel5 = selector.transform(X)
print(X_sel5)

[[25  0  0  0  0]
 [25  0  0  0  0]
 [25  0  0  0  0]
 ...
 [46  0  0  0  0]
 [27  0  0  1  0]
 [25  0  0  1  0]]


**----여기까지 추가----**

#### 결정 트리 모델의 작성과 검증

In [22]:
# Scikit-learn의 로드
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold교차 검증
kf = KFold(n_splits=10, shuffle=True)
# 모델의 정밀도를 보존하기 위한 준비
scores = []
# 데이터를 셔플하고, 훈련데이터와 테스트 데이터를 분활
for train_id, test_id in kf.split(X_sel5):
    # 훈련 데이터를 사용한 모델을 작성
    x = X_sel5[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)
    # 테스트 데이터에 모델을 적용
    pred_y = clf.predict(X_sel5[test_id])
    # 모델의 정밀도를 계산하고 보존
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# 모델의 평균정밀도, 표준편차를 확인
scores = np.array(scores)
print(scores.mean(), scores.std())

0.9632952299556647 0.00845802437294304


In [23]:
# Scikit-learn의 로드
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 재현률, 적합률의 확인
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.9306930693069307
0.9791666666666666


#### 모델의 정밀도 향상

In [24]:
# 모델작성 패러메터의 확인
print(clf)

DecisionTreeClassifier()


In [25]:
# Scikit-learn의 로드
from sklearn.model_selection import GridSearchCV

# 패러메터의 범위설정
params = {
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# 그리드 서치의 실행조건을 설정
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, 
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')

# 그리드 서치 실행
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [27]:
# 가장 높은 정밀도와 패러메터의 조합을 표시
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.9689893702259494
{'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 10}
